# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from pandasql import sqldf

In [2]:
mysql = lambda q : sqldf(q , globals())

## Uploading Training Data

In [3]:
df_train = pd.read_csv('../input/jobathon-data-set/train_MpHjUjU.csv')
df_train.info()

## Converting Data into useable format

In [4]:
main = mysql('''select Emp_ID , max(Age) as Age , Gender , City ,Education_Level , avg(Salary) as Salary , Dateofjoining,
avg(Designation) as Designation , avg(Joining_Designation) as Joining_Designation,
avg(Quarterly_Rating) as Quarterly_Rating , sum(Total_Business_Value) as Total_Business_Value
from df_train
group by Emp_ID , Gender , City ,Education_Level ,Dateofjoining
''')

In [5]:
LWD = mysql('''select Emp_ID , LastWorkingDate
from df_train
where LastWorkingDate is not null
''')

In [6]:
Neg_BV = mysql('''select Emp_ID , Total_Business_Value
from df_train
where Total_Business_Value < 0
group by Emp_ID 
''')

In [7]:
c = mysql('''select main.Emp_ID , Age , Gender , City ,Education_Level ,Salary , Dateofjoining, Designation ,
Joining_Designation, Quarterly_Rating , Total_Business_Value ,LastWorkingDate 
from main
left join LWD
on main.Emp_ID = LWD.Emp_ID
''')

In [8]:
df_train = mysql('''select c.Emp_ID , Age , Gender , City ,Education_Level ,Salary , Dateofjoining, Designation ,
Joining_Designation, Quarterly_Rating , c.Total_Business_Value ,LastWorkingDate , Neg_BV.Total_Business_Value as neg
from c
left join Neg_BV
on c.Emp_ID = Neg_BV.Emp_ID
''')

In [9]:
df_train.info()

# Feature Engineering

In [10]:
df_train['TBV_negative'] = df_train['neg'].apply(lambda x: 1 if x < 0 else 0) #if they sold anytime

In [11]:
df_train.neg.fillna(1 , inplace = True)

In [12]:
df_train.info()

In [13]:
df_train.LastWorkingDate.fillna(0 , inplace = True)

In [14]:
df_train['target'] = df_train['LastWorkingDate'].apply(lambda x: 0 if x == 0 else 1)

In [15]:
df_train

In [16]:
df_train.target.value_counts()

# Label Encoder

In [17]:
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()

In [18]:
cat_feat = ['Gender', 'City', 'Education_Level' ,'Dateofjoining' , 'LastWorkingDate']

In [19]:
df_train[cat_feat] = df_train[cat_feat].astype(str).apply(lbl_enc.fit_transform)
df_train

# Train Test Split

In [20]:
df_train.columns.values

In [21]:
a = df_train.loc[: , ['Emp_ID', 'Age', 'Gender', 'City', 'Education_Level', 'Salary',
       'Dateofjoining', 'Designation', 'Joining_Designation',
       'Quarterly_Rating', 'Total_Business_Value',
       'neg', 'TBV_negative']]
b = df_train.loc[: ,'target']
a_train , a_test , b_train , b_test = train_test_split( a, b, test_size = 0.05 , random_state = 0 )
a_train.shape , a_test.shape

# DecisionTree

In [22]:
from sklearn import tree

In [23]:
classifierDT = tree.DecisionTreeClassifier(max_depth=None, min_samples_leaf=2, min_samples_split = 5 ,
    ccp_alpha = 0.001 , random_state = 2 )
classifierDT.fit(a_train , b_train)
b_predDT = classifierDT.predict(a_test)
F1DT = metrics.f1_score(b_test , b_predDT)
F1DT

# XGB

In [24]:
import xgboost as xgb

In [25]:
xg = xgb.XGBClassifier(n_estimators = 170 ,random_state = 1 , learning_rate = 0.01)
xg.fit(a_train , b_train)
b_predxg = xg.predict(a_test)
F1xg = metrics.f1_score(b_test , b_predxg)
F1xg

In [26]:
pd.DataFrame(
            metrics.confusion_matrix(b_test ,b_predxg),
            columns= ['passenger not S' , 'passenger S'],
            index = ['true not survive' , 'true S'])

# Eli5

In [27]:
col = a_train.columns.to_list()

In [28]:
import eli5
eli5.show_weights(classifierDT , feature_names = col)

# SHAP Analysis

In [29]:
import shap
shap.initjs()

In [30]:
explainer = shap.TreeExplainer(xg)
shap_values = explainer.shap_values(a_train)

In [31]:
shap.summary_plot( shap_values , features = a_train , feature_names = col)

## Uploading Testing Data

In [32]:
df_test = pd.read_csv('../input/jobathon-test-data/test_hXY9mYw.csv')
df_test

In [33]:
df_test = mysql('''select  d.Emp_ID, d.Age, Gender, City, Education_Level, Salary,
       Dateofjoining, Designation, Joining_Designation,
       Quarterly_Rating, Total_Business_Value, LastWorkingDate,
       neg, TBV_negative, target
from df_train as d
inner join df_test as dt
on d.Emp_id = dt.Emp_id
''')
df_test

In [34]:
df_test.columns.values

In [35]:
att = df_test.loc[: , ['Emp_ID', 'Age', 'Gender', 'City', 'Education_Level', 'Salary',
       'Dateofjoining', 'Designation', 'Joining_Designation',
       'Quarterly_Rating', 'Total_Business_Value',
       'neg', 'TBV_negative']].values

In [36]:
b = xg.predict(att)

## Creating Final Submission File

In [37]:
finalSub = pd.DataFrame(df_test['Emp_ID'])
finalSub

In [38]:
finalSub['Target'] = b 

In [39]:
finalSub

In [40]:
finalSub.Target.value_counts()

In [41]:
from IPython.display import HTML
import base64

In [42]:
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [43]:
create_download_link(finalSub)